# Overview of this notebook:

1. convert to two class (c-/abshort &c+/ablong) and the holdout c-/ablong
2. split the train and test set
3. median standardize
4. aggregate
5. impute

i do 1-3 BEFORE I AGGREGATE and median standardize, as some information may be leaking from the train/test set as i had it previously.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
import numpy as np
import glob

from sklearn.externals.joblib import Memory
from sklearn.model_selection import train_test_split


%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

In [2]:
#patients of interest from rotation_cohort_generation
from parameters import final_pt_df_v, date, repository_path, categorical

#patients of interest from rotation_cohort_generation
final_pt_df2 = final_pt_df_v #pd.read_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_final_pt_df2.csv'%(most_updated_patient_df), index_col=0)
del(final_pt_df_v)

patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]


time: 973 ms


In [3]:
len(patients)

15412

time: 3.75 ms


In [4]:

def save_df(df, df_name='default', add_subfolder=False):
    #uses the date and supplied df name and saves to the savepath specified above.
    
    save_path= str(repository_path)+'/data/cleaned_merged_agg/'
    if df_name == 'default':
        df_name= "%s"%(df)
    
    address=save_path+'%s/'%(folder)
    if not os.path.exists(address):
        print(address)
        os.makedirs(address)
    pd.DataFrame(df).to_csv(Path(address+'%s_%s_cleaned_merged_agg.csv' %(date, df_name)))

time: 1.48 ms


In [5]:
from parameters import lower_window, upper_window, folder, date, time_col, time_var, patient_df


time: 565 µs


In [6]:
#importing cleaned_merged big_df
allFiles = glob.glob(str(repository_path)+ '/data/cleaned_merged/%s/'%(folder) + "{}_*.csv".format(date))

df_list=[]
for element in allFiles:
    df_list.append(element.split('{}_'.format(date))[1].split('_prepped.csv')[0]) #making an list of all my dataframes in order they appear in file

i=0
for name in df_list:
    big_df = pd.read_csv(allFiles[i],  index_col=0)

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 5.35 s


In [7]:
#quick housekeeping addition to accomidate older generated data
if len(big_df.loc[big_df['label']=="pao2/fio2",'label'])>1:
    big_df.loc[big_df['label']=="pao2/fio2",'label']="pao2fio2ratio"

time: 220 ms


In [8]:
big_df.head()

,icustay_id,t_0,label,value,delta,uom,source,subject_id
502618,200001.0,2181-11-26,bands,absent,0 days 00:00:00.000000000,y/n,labs,55973
42164,200001.0,2181-11-26,pco2,absent,0 days 00:00:00.000000000,y/n,bg_ART,55973
15252,200001.0,2181-11-26,any_vasoactives,0.0,0 days 00:00:00.000000000,y/n,all_vaso,55973
14358,200001.0,2181-11-26,cancer_elix,0.0,0 days 00:00:00.000000000,y/n,cancer_elix,55973
17136,200001.0,2181-11-26,daily_sofa,5,0 days 00:00:00.000000000,daily_sofa_score,sofa,55973


time: 18 ms


In [9]:
#overview of all variables and formats
big_df.groupby('label')['value'].describe().sort_values('unique')

,count,unique,top,freq
label,,,,
any_vasoactives,19633,2,0.0,13919
leukocyte,19633,2,Neg/Not_tested,17390
nitrite,19633,2,Neg/Not_tested,19187
norepinephrine,19633,2,0.0,16335
o2_flow,19633,2,0.0,19022
gender,19633,2,M,10753
epinephrine,19633,2,0.0,19236
vasopressin,19633,2,0.0,19251
dobutamine,19633,2,0.0,19408


time: 1.29 s


## initial data prep
* convert to two class

In [10]:
#convert to two class
final_pt_df2['final_bin'].unique()
two_classes=['C_neg/A_partial','C_pos/A_full']
two_class_icu=final_pt_df2.loc[final_pt_df2.loc[:,"final_bin"].isin(two_classes),['icustay_id','subject_id','final_bin']]


time: 7.98 ms


In [11]:
# hold out class
ho_class=['C_neg/A_full']
one_class_icu=final_pt_df2.loc[final_pt_df2.loc[:,"final_bin"].isin(ho_class),['icustay_id','subject_id','final_bin']]


time: 5.29 ms


In [12]:
big_df_ho=big_df.loc[big_df['icustay_id'].isin(list(one_class_icu['icustay_id'])),:].copy()




time: 684 ms


In [13]:
big_df=big_df.loc[big_df['icustay_id'].isin(list(two_class_icu['icustay_id'])),:].copy()

time: 730 ms


In [14]:
len(big_df)
big_df['icustay_id'].nunique() 

10305

time: 25.7 ms


### roundabout way of sampling train/test set so that each subject is only in either train or test:
* To ensure a single patient did not end up with samples in both training and testing sets, individual patients (subject_id's) are kept together when performing the stratified train and test split

In [15]:
# label each subject_id with the max of the two classes. 
two_class_pt=two_class_icu.copy()
two_class_pt['final_bin']=pd.factorize(two_class_pt['final_bin'])[0]
two_class_maxsub=two_class_pt.loc[two_class_pt.groupby('subject_id')['final_bin'].idxmax(),:]

time: 1.29 s


In [16]:
# 70/30 train/test set split with 12345=seed, splitting on max final bin of each SUBJECT_ID
train, test = train_test_split(two_class_maxsub, test_size=0.3, random_state=12345, 
                               stratify=two_class_maxsub['final_bin'])
# generate list of each SUBJECT_ID in each split
train_subject=list(train['subject_id'])
test_subject=list(test['subject_id'])

#filtering big_df on train subjects and test subjects to get my train/test splits.
big_df_train= big_df.loc[big_df.loc[:,'subject_id'].isin(train_subject),:].copy()
big_df_test= big_df.loc[big_df.loc[:,'subject_id'].isin(test_subject),:].copy()
del big_df

time: 723 ms


## calc median/iqr of 'healthiest patients' for standardization
take all non-categorical variables for HEALTHY PATIENTS and calculate the median and IQR for them. then will use this to make z scores via:

$$Z=\frac{(X-\widetilde{X}_{-/short})}{(IQR_{-/short})}$$ where $\widetilde{X}_{-/short}$ is the median value of the patients with negative SSC and short duration EAT.


In [17]:
big_df_train[big_df_train['label']=='vent_recieved']['icustay_id'].nunique()

7111

time: 101 ms


In [18]:
big_df_train['icustay_id'].nunique()

7205

time: 17.4 ms


In [19]:
from parameters import continuous, onetime


time: 1.07 ms


In [50]:
def median_label_fxn(big_df):
    """ grabs the median and IQR values for all continuous variables based upon the values present in
    the last 24 hours of the c_neg/A_partial patients (AKA the healthiest patients at the healthiest timepoints) """
    
    global final_pt_df2
    healthy_pt=list(final_pt_df2[final_pt_df2['final_bin']=="C_neg/A_partial"]['icustay_id'])

    #filter to only healthy patients filter
    ##splitting big_df, making a copy and restricting it to all values that will be usd in standardizing
    healthy_df=big_df[big_df['label'].isin(continuous+onetime)].copy()
    healthy_df['value']= pd.to_numeric(healthy_df['value']) #converting to numeric 
    healthy_df=healthy_df[healthy_df['icustay_id'].isin(healthy_pt)].copy() #only numerical values for cneg/ab partial pt
    
    #finding the last 24 hours of each healthy patient.
    healthy_pt_end=pd.DataFrame(healthy_df.groupby("icustay_id")['delta'].max())
    healthy_pt_end["start"]=healthy_pt_end['delta']- pd.to_timedelta("1 day 00:00:00")
    healthy_pt_end=healthy_pt_end.rename(columns={"delta":"end"}).reset_index()
    healthy_df= pd.merge(healthy_df,healthy_pt_end, left_on="icustay_id", right_on="icustay_id", how="left" ) #now have the last 24 hours annotated for each patient as start and end.
    
    #calculating medians and iqr for each label based on healthy patient's last 24 hours in icu
    median_label=pd.DataFrame((healthy_df.groupby("label")['value'].median())).reset_index()
    median_label=median_label.rename(columns={'value':"median"})
    iqr_label=pd.DataFrame((healthy_df.groupby("label")['value'].quantile(0.75)-healthy_df.groupby("label")['value'].quantile(0.25))).reset_index()
    iqr_label=iqr_label.rename(columns={'value':"iqr"})
    median_label=pd.merge(median_label,iqr_label)  #final median df
    return(median_label)

def standardization_fxn(big_df, big_df_2class=None): ##changed here
    global continuous, onetime, vaso_active, ordinal, categorical
    
    if big_df_2class is not None:
        median_label=median_label_fxn(big_df_2class)
    else:
        median_label=median_label_fxn(big_df)
    
    ### dataformatting: convert all dtypes to a numeric type that pereserves nan. 
    #splitting categorical, ordinal and continuous
    big_categorical= big_df.loc[big_df.loc[:,'label'].isin(categorical),:].copy() 

    #continuous and ordinal variables
    big_noCat= big_df.loc[big_df.loc[:,'label'].isin(continuous+onetime),:].copy() 
    big_noCat['value']= big_noCat['value'].apply(pd.to_numeric, args=('coerce',)) #instead of convert to float, may preserve nan's better. 
    
    ### adding a standardized value (x-median)/iqr  where median is of the last 24 hours in time window for culture neg/ ab partial patients 
    big_noCat=pd.merge(big_noCat, median_label, how="left") 
    #loging values
    big_noCat['median']= np.log(big_noCat['median']+1.0)
    big_noCat['iqr']= np.log(big_noCat['iqr']+1.0)
    big_noCat['raw_value']=big_noCat['value']
    big_noCat['value']=np.log(big_noCat['value']+1.0)
    
    big_noCat['standardize']=((big_noCat['value']-big_noCat['median'])/big_noCat['iqr']).fillna(0) #standardize is log standardized  

    return(big_noCat, big_categorical)

time: 4.03 ms


In [21]:
noCat_df_train, cat_df_train = standardization_fxn(big_df_train)
noCat_df_test, cat_df_test = standardization_fxn(big_df_test)

time: 52 s


In [51]:
##### holdout
noCat_df_ho, cat_df_ho = standardization_fxn(big_df_ho, big_df_train)


time: 40.4 s


In [23]:
#should be same number of col as len(categorical)
cat_df_train[cat_df_train['icustay_id']==200012.0].groupby('label')['value'].value_counts()#.head(10)

label            value         
any_vasoactives  0.0               1
bands            >10               1
cancer_elix      0.0               1
dobutamine       0.0               1
dopamine         0.0               1
epinephrine      0.0               1
ethnicity        asian             1
gender           F                 1
leukocyte        Neg/Not_tested    1
nitrite          Neg/Not_tested    1
norepinephrine   0.0               1
o2_flow          0.0               1
pco2             absent            1
phenylephrine    0.0               1
rrt              0.0               1
vasopressin      0.0               1
vent_recieved    Oxygen            1
Name: value, dtype: int64

time: 10 ms


In [54]:
noCat_df_ho[noCat_df_ho['label']=='weight'].head()

,icustay_id,t_0,label,value,delta,uom,source,subject_id,median,iqr,raw_value,standardize
6,200001.0,2181-11-26,weight,4.127134,0 days 00:00:00.000000000,kg,weight,55973,4.35735,3.317816,61.0,-0.069388
191,200003.0,2199-08-04,weight,4.356709,0 days 00:00:00.000000000,kg,weight,27513,4.35735,3.317816,77.0,-0.000193
441,200019.0,2178-07-09,weight,4.189655,0 days 00:00:00.000000000,kg,weight,21789,4.35735,3.317816,65.0,-0.050544
625,200065.0,2120-03-23,weight,4.795791,0 days 00:00:00.000000000,kg,weight,19891,4.35735,3.317816,120.0,0.132147
952,200066.0,2188-10-07,weight,4.369448,0 days 00:00:00.000000000,kg,weight,214,4.35735,3.317816,78.0,0.003646


time: 94.3 ms


In [55]:
noCat_df_ho[noCat_df_ho['label']=='lactate'].head()

,icustay_id,t_0,label,value,delta,uom,source,subject_id,median,iqr,raw_value,standardize
230,200003.0,2199-08-04,lactate,0.916291,0 days 01:32:00.000000000,mmol/L,labs,27513,1.029619,0.955511,1.5,-0.118605
635,200065.0,2120-03-23,lactate,2.397895,0 days 01:16:00.000000000,mmol/L,labs,19891,1.029619,0.955511,10.0,1.431983
653,200065.0,2120-03-23,lactate,2.054124,0 days 03:04:00.000000000,mmol/L,labs,19891,1.029619,0.955511,6.8,1.072205
672,200065.0,2120-03-23,lactate,2.397895,0 days 05:22:00.000000000,mmol/L,labs,19891,1.029619,0.955511,10.0,1.431983
704,200065.0,2120-03-23,lactate,2.484907,0 days 08:28:00.000000000,mmol/L,labs,19891,1.029619,0.955511,11.0,1.523045


time: 97.8 ms


In [24]:
noCat_df_train[noCat_df_train['label']=='lactate']

,icustay_id,t_0,label,value,delta,uom,source,subject_id,median,iqr,raw_value,standardize
5,200012.0,2153-12-23,lactate,0.832909,0 days 02:36:00.000000000,mmol/L,labs,28448,1.029619,0.955511,1.3,-0.205869
135,200014.0,2105-02-17,lactate,0.641854,0 days 01:08:00.000000000,mmol/L,labs,9514,1.029619,0.955511,0.9,-0.405820
390,200033.0,2198-08-11,lactate,0.955511,0 days 05:16:00.000000000,mmol/L,labs,56369,1.029619,0.955511,1.6,-0.077558
432,200033.0,2198-08-11,lactate,0.832909,0 days 09:36:00.000000000,mmol/L,labs,56369,1.029619,0.955511,1.3,-0.205869
709,200055.0,2179-03-17,lactate,0.916291,0 days 08:12:00.000000000,mmol/L,labs,25225,1.029619,0.955511,1.5,-0.118605
...,...,...,...,...,...,...,...,...,...,...,...,...
1496964,299863.0,2154-12-15,lactate,0.875469,0 days 15:46:00.000000000,mmol/L,labs,31011,1.029619,0.955511,1.4,-0.161328
1496991,299863.0,2154-12-15,lactate,0.875469,0 days 18:04:00.000000000,mmol/L,labs,31011,1.029619,0.955511,1.4,-0.161328
1497035,299863.0,2154-12-15,lactate,1.029619,0 days 22:44:00.000000000,mmol/L,labs,31011,1.029619,0.955511,1.8,0.000000
1497283,299903.0,2111-06-18,lactate,0.587787,0 days 03:04:00.000000000,mmol/L,labs,31252,1.029619,0.955511,0.8,-0.462404


time: 515 ms


# aggregation:
### clincally guided min/max

In [56]:
from parameters import low_value, hi_value, both_value

# important_onetime=['yearsold','weight'] #pco2 and bands are now in categorical 12-12-19


time: 453 µs


In [57]:
noCat_df_train['label'].unique()#noCat_df_train['label']=='any_vasoactives'

array(['daily_sofa', 'pao2fio2ratio', 'sum_elix', 'weight', 'yearsold',
       'lactate', 'mingcs', 'diasbp', 'heartrate', 'meanartpress',
       'resprate', 'spo2', 'sysbp', 'temperature', 'hemoglobin',
       'platelet', 'wbc', 'calcium', 'glucose', 'ph', 'bicarbonate',
       'bun', 'chloride', 'creatinine', 'inr', 'potassium', 'ptt',
       'sodium', 'bilirubin'], dtype=object)

time: 58.1 ms


### running the min/max aggregations.



In [58]:
def clin_agg(big_noCat, big_cat, big_df, values="standardize"):
    """
    clincally guided aggregations.
    
    values= choose here if want to use standardization or raw values.
    
    note: getting two minor errors, could use some cleaning up at later date.
    """
    global hi_value, low_value, both_value, important_onetime
    #max aggregation for selected variables
    big_max= big_noCat.loc[big_noCat.loc[:,'label'].isin(hi_value),:]
    table = pd.pivot_table(big_max, values=values, columns='label', index=['icustay_id'],aggfunc=max, dropna=False)
    
    #min aggregation for selected variables
    big_min= big_noCat.loc[big_noCat.loc[:,'label'].isin(low_value),:]
    table2 = pd.pivot_table(big_min, values=values, columns='label', index=['icustay_id'],aggfunc=min, dropna=False)
    
    #max&min aggregation for selected variables
    big_both= big_noCat.loc[big_noCat.loc[:,'label'].isin(both_value),:]
    table3 = pd.pivot_table(big_both, values=values, columns='label', index=['icustay_id'],aggfunc=[max,min], dropna=False)
    
    #first left join all different continuous aggregations together. 
    worst_df=pd.merge(table.reset_index(), table2.reset_index(), how='left')
    worst_df=pd.merge(worst_df, table3.reset_index(), left_on='icustay_id', right_on='icustay_id',how='left')
    
    ### formatting categorical to wide format to match the tables/worst_df
    big_cat= big_cat.pivot(
    index='icustay_id',
    values='value',
    columns='label').reset_index() #need to convert to wide format. should be one row per icustay per time. 
    
    ## merging the categorical and aggregated dataframes together. 
    worst_df=pd.merge(worst_df, big_cat, left_on='icustay_id', right_on='icustay_id',how='left') #using max/min aggregates 

#     #adding important one_time values to final aggregated
#     agg_remaining= big_df.loc[big_df.loc[:,'label'].isin(important_onetime),:]
#     agg_table2 = pd.pivot_table(agg_remaining, values='value', columns='label', index=['icustay_id'],aggfunc=[max], dropna=False) 
#     agg_table2.columns = agg_table2.columns.get_level_values(1)
#     agg_table2=agg_table2.reset_index()
#     agg_table2.head()#.rename(columns={})

#     worst_df=pd.merge(worst_df, agg_table2, how='left')
    return(worst_df)


time: 2.61 ms


In [32]:
hi_value

{'bilirubin',
 'bun',
 'chloride',
 'creatinine',
 'daily_sofa',
 'glucose',
 'heartrate',
 'height',
 'inr',
 'lactate',
 'potassium',
 'ptt',
 'resprate',
 'sum_elix',
 'temperature',
 'weight',
 'yearsold'}

time: 1.88 ms


In [59]:
worst_df_train=clin_agg(noCat_df_train, cat_df_train,big_df_train, values="standardize")
worst_df_test=clin_agg(noCat_df_test, cat_df_test, big_df_test, values="standardize")
worst_df_ho=clin_agg(noCat_df_ho, cat_df_ho,big_df_ho, values="standardize")

time: 1.79 s


In [40]:
worst_df_train.head()

,icustay_id,bilirubin,bun,chloride,creatinine,daily_sofa,glucose,heartrate,inr,lactate,...,gender,leukocyte,nitrite,norepinephrine,o2_flow,pco2,phenylephrine,rrt,vasopressin,vent_recieved
0,200012.0,NaN,NaN,NaN,NaN,-0.430677,NaN,0.072874,NaN,-0.205869,...,F,Neg/Not_tested,Neg/Not_tested,0.0,0.0,absent,0.0,0.0,0.0,Oxygen
1,200014.0,NaN,0.040770,0.037089,-0.300758,-0.178747,-0.053798,0.017360,-0.270369,-0.405820,...,M,Neg/Not_tested,Neg/Not_tested,0.0,0.0,<50,0.0,0.0,0.0,Mech
2,200033.0,NaN,-0.082229,-0.004314,-0.219047,-0.430677,0.136180,0.092040,-0.132111,-0.077558,...,M,Neg/Not_tested,Neg/Not_tested,0.0,0.0,absent,1.0,0.0,0.0,Mech
3,200036.0,NaN,NaN,NaN,NaN,-0.430677,-0.021339,0.127153,NaN,NaN,...,M,Neg/Not_tested,Neg/Not_tested,0.0,0.0,absent,0.0,0.0,0.0,Oxygen
4,200055.0,-0.316061,0.098917,-0.035714,1.934240,0.503859,-0.029584,0.049416,-0.132111,-0.118605,...,F,Neg/Not_tested,Neg/Not_tested,0.0,0.0,absent,0.0,1.0,0.0,None


time: 20.2 ms


In [60]:
worst_df_ho['pao2fio2ratio'].describe()

count    7401.000000
mean        0.040561
std         0.119575
min        -0.475168
25%        -0.052025
50%         0.086184
75%         0.145556
max         0.302855
Name: pao2fio2ratio, dtype: float64

time: 5.62 ms


In [61]:
worst_df_ho['weight'].describe()

count    6688.000000
mean        0.004226
std         0.085967
min        -1.104402
25%        -0.050544
50%         0.002500
75%         0.054114
max         0.666066
Name: weight, dtype: float64

time: 5.88 ms


In [41]:
worst_df_train['pao2fio2ratio'].describe()

count    7205.000000
mean        0.076559
std         0.104366
min        -0.354458
25%         0.000000
50%         0.145556
75%         0.145556
max         0.335424
Name: pao2fio2ratio, dtype: float64

time: 6.29 ms


In [42]:
worst_df_train['vasopressin'].value_counts()

0.0    7109
1.0      96
Name: vasopressin, dtype: int64

time: 4.96 ms


In [43]:
#worst_df_train['vasopressin'].value_counts()

time: 380 µs


In [44]:
worst_df_train['bands'].value_counts()

absent    6418
<10        485
>10        302
Name: bands, dtype: int64

time: 4.76 ms


In [45]:
worst_df_train['pco2'].value_counts()

absent    4892
<50       1709
>50        604
Name: pco2, dtype: int64

time: 5.01 ms


In [64]:
save_df(worst_df_train, 'train')
save_df(worst_df_test, 'test')
save_df(worst_df_ho, 'longneg')

time: 1.1 s


In [36]:
del worst_df_train, worst_df_test

time: 6 ms
